In [ ]:
import time
from config_ko import DB_CONFIG, ACCOUNT_INFO
from functions import check_account, get_auth_info, get_current_price, execute_order
import os, datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

BASE_DIR   = "results_3"
CSV_PATH   = os.path.join(BASE_DIR, "account_log.csv")
OUT_EQUITY = os.path.join(BASE_DIR, "총평가금액_추세.png")
OUT_RET    = os.path.join(BASE_DIR, "수익률_추세.png")
os.makedirs(BASE_DIR, exist_ok=True)
START_CAPITAL = 100_000_000  # 초기 투자금
app_key, app_secret, access_token = get_auth_info()

🟢 기존 토큰 재사용 중 (파일)


In [ ]:
portfolio, account = check_account(access_token, app_key, app_secret)
portfolio.loc[0] = ['005930', 10, 70000]   # 삼성전자
portfolio.loc[1] = ['000660', 5, 280000]   # SK하이닉스
portfolio.loc[2] = ['035720', 3, 40000]   # 카카오
portfolio['현재가'] = portfolio['종목코드'].apply(
    lambda code: (time.sleep(0.5), get_current_price(access_token, app_key, app_secret, code))[1]
)
portfolio['변동률'] = (portfolio['현재가'] - portfolio['매입단가']) / portfolio['매입단가'] * 100
print(portfolio)
print()
print("총평가금액:",int(account['tot_evlu_amt']))  
print("총수익률:", (int(account['tot_evlu_amt'])-100000000)/1000000)
print("매수금액:", int(account['pchs_amt_smtl_amt']))
print("현재 평가금액:", int(account['evlu_amt_smtl_amt']))
print("평가 손익:", int(account['evlu_pfls_smtl_amt']))
print("예수금:", int(account['tot_evlu_amt']) - int(account['scts_evlu_amt']))


📡 응답 상태코드: 200
     종목코드  보유수량    매입단가     현재가        변동률
0  005930    10   70000   72600   3.714286
1  000660     5  280000  301500   7.678571
2  035720     3   40000   59800  49.500000

총평가금액: 102980558
총수익률: 2.980558
매수금액: 0
현재 평가금액: 0
평가 손익: 0
예수금: 102980558


In [139]:
for idx, row in portfolio.iterrows():

    if row['변동률'] >= 3 or row['변동률'] <= -3:
        print(row['종목코드'])
        #execute_order(row['종목코드'], row['보유수량'], "매도", "시장가", app_key, app_secret, access_token)


005930
000660
035720


In [ ]:
def log_account(account: dict):
    today = dt.date.today().isoformat()
    tot_evlu_amt = int(account['tot_evlu_amt'])
    tot_ret = (tot_evlu_amt - START_CAPITAL) / 1_000_000
    pchs_amt = int(account['pchs_amt_smtl_amt'])
    cur_evlu_amt = int(account['evlu_amt_smtl_amt'])
    pnl = int(account['evlu_pfls_smtl_amt'])
    cash = int(account['tot_evlu_amt']) - int(account['scts_evlu_amt'])

    row = [today, tot_evlu_amt, tot_ret, pchs_amt, cur_evlu_amt, pnl, cash]

    file_exists = os.path.exists(CSV_PATH)
    with open(CSV_PATH, "a", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(["날짜","총평가금액","총수익률","매수금액","현재평가금액","평가손익","예수금"])
        writer.writerow(row)

    print(f"계좌 정보 저장 완료: {row}")
log_account(account)

In [164]:
# 기간 설정(포함 범위)
RANGE_START = "2025-09-01"
RANGE_END   = "2025-10-03"

def plot_equity_and_return_from_csv():
    if not os.path.exists(CSV_PATH):
        print("CSV가 없습니다:", CSV_PATH)
        return

    import matplotlib.dates as mdates

    # 폰트(윈도우 기준)
    plt.rcParams['font.family'] = 'Malgun Gothic'
    plt.rcParams['axes.unicode_minus'] = False

    # 1) 데이터 읽기
    df = pd.read_csv(CSV_PATH, encoding="utf-8-sig")
    if "날짜" not in df.columns or "총평가금액" not in df.columns:
        raise ValueError("CSV에 '날짜', '총평가금액' 컬럼이 있어야 합니다.")

    df["날짜"] = pd.to_datetime(df["날짜"])
    df = df.sort_values("날짜").drop_duplicates(subset=["날짜"], keep="last")

    # 2) 기간 필터 (포함)
    start = pd.Timestamp(RANGE_START)
    end   = pd.Timestamp(RANGE_END)
    mask = (df["날짜"] >= start) & (df["날짜"] <= end)
    df = df.loc[mask].copy()
    if df.empty:
        print(f"선택된 기간({RANGE_START} ~ {RANGE_END})에 데이터가 없습니다.")
        return

    # 3) 일단위 리샘플(결측일은 직전값으로 채움)
    df = (df.set_index("날짜")
            .resample("D")
            .last()
            .ffill()
            .reset_index())

    # 4) 수익률 파생
    df["수익률_%"]  = (df["총평가금액"] / START_CAPITAL - 1.0) * 100.0
    df["수익률_백만"] = (df["총평가금액"] - START_CAPITAL) / 1_000_000

    # 마지막(오늘) 값
    last_date   = df["날짜"].iloc[-1]
    last_equity = int(df["총평가금액"].iloc[-1])
    last_ret_pct = df["수익률_%"].iloc[-1]
    last_ret_mil = df["수익률_백만"].iloc[-1]

    # X축 포맷러(날짜 간결 표시)
    locator = mdates.AutoDateLocator()
    formatter = mdates.ConciseDateFormatter(locator)

    # ── [A] 총평가금액 ──
    fig = plt.figure(figsize=(11, 6.5))
    fig.suptitle(
        f"총평가금액 추세 ({RANGE_START} ~ {RANGE_END}) | "
        f"마지막 {last_date.date()}  총평가금액 {last_equity:,.0f}원  "
        f"수익률 {last_ret_pct:+.2f}% ({last_ret_mil:+.2f} 백만)",
        fontsize=13, fontweight="bold", y=0.98
    )
    ax = plt.gca()
    ax.plot(df["날짜"], df["총평가금액"], marker="o", linewidth=1.8, label="총평가금액")
    ax.scatter([last_date], [last_equity], s=50)
    ax.text(last_date, last_equity, f" {last_equity:,.0f}원",
            va="bottom", ha="left", fontsize=10)

    ax.grid(True, alpha=0.35)
    ax.set_xlabel("날짜")
    ax.set_ylabel("총평가금액 (원)")
    ax.set_xlim(start, end)
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_major_formatter(formatter)

    plt.tight_layout(rect=[0, 0, 1, 0.93])
    out_equity = os.path.join(BASE_DIR, f"총평가금액_추세_{RANGE_START}_{RANGE_END}.png")
    plt.savefig(out_equity, dpi=200)
    plt.close()
    print("저장:", out_equity)

    # ── [B] 수익률(%) ──
    plt.figure(figsize=(11, 6.5))
    plt.title(f"수익률(%) 추세 ({RANGE_START} ~ {RANGE_END})")
    plt.plot(df["날짜"], df["수익률_%"], marker="o", linewidth=1.8, label="수익률(%)")
    plt.scatter([last_date], [last_ret_pct], s=50)
    plt.text(last_date, last_ret_pct, f" {last_ret_pct:+.2f}%",
             va="bottom", ha="left", fontsize=10)

    plt.grid(True, alpha=0.35)
    plt.xlabel("날짜")
    plt.ylabel("수익률 (%)")
    plt.xlim(start, end)
    ax = plt.gca()
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_major_formatter(formatter)

    plt.tight_layout()
    out_ret = os.path.join(BASE_DIR, f"수익률_추세_{RANGE_START}_{RANGE_END}.png")
    plt.savefig(out_ret, dpi=200)
    plt.close()
    print("저장:", out_ret)
plot_equity_and_return_from_csv()

저장: results_3\총평가금액_추세_2025-09-01_2025-10-03.png
저장: results_3\수익률_추세_2025-09-01_2025-10-03.png
